In [1]:
%load_ext autoreload
%autoreload 1
import pandas as pd

%aimport pyscisci
import random

In [2]:
pyscisci

<module 'pyscisci' from '/Users/ajgates/Documents/GitHub/pyscisci/pyscisci/__init__.py'>

In [3]:
pyscisci.analysis.groupby_range

<function pyscisci.analysis.groupby_range(df, colgroupby, colrange)>

In [4]:
pub2ref = pd.DataFrame([[0,1], [0,2], [0,3],[1,4],[1,5], [2,5], [3,1], [10,1], [7,1]], 
                       columns = ['CitingPublicationId', 'CitedPublicationId'])

reference_groups = pub2ref.groupby('CitingPublicationId', as_index = False, sort = False)['CitedPublicationId']
citation_groups = pub2ref.groupby('CitedPublicationId', sort = False)['CitingPublicationId']

    
reference_groups.groups.get(1)

Int64Index([3, 4], dtype='int64')

In [52]:
from functools import reduce

def get_citation_groups(pid):
    try:
        return citation_groups.get_group(pid).values
    except KeyError:
        return []
    
def disruption_index(citing_focus):
    focusid = citing_focus.name
    
    # if the focus publication has no references, then it has a disruption of 1.0
    try:
        focusref = reference_groups.get_group(focusid)
    except KeyError:
        return 1.0
    
    cite2ref = reduce(np.union1d, [get_citation_groups(refid) for refid in focusref.values])
    
    nj = np.intersect1d(cite2ref, citing_focus.values).shape[0]
    ni = citing_focus.shape[0] - nj
    nk = cite2ref.shape[0] - nj
    return (ni - nj)/(ni + nj + nk)

citation_groups.apply(disruption_index)

CitedPublicationId
1    0.666667
2    0.333333
3   -0.250000
4    1.000000
5    1.000000
Name: CitingPublicationId, dtype: float64

In [42]:
import numpy as np

anp = np.arange(1000)
aset = set([i for i in range(1000)])
bnp = np.arange(500, 5000)
bset = set([i for i in range(500, 5000)])

In [39]:
%timeit np.intersect1d(anp, bnp)

137 µs ± 5.09 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [43]:
%timeit aset & bset

20.5 µs ± 1.45 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [45]:
%timeit np.isin(anp, bnp)

146 µs ± 6.91 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [46]:
%timeit [a in bset for a in aset]

74.2 µs ± 1.72 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [7]:
for pid in range(Npubs):
    pub = database.get_pub(pid)
    print(pub.id, pub.year, pub.ck(4), pub.c10, pub.total_citations, pub.disruption_index)
    print("references: {0} | citations: {1}".format(pub.references, pub.citations))
    print(pub.cocited_publications)
    print(pub.yearly_cnorm)
    print()

0 2000 4 9 9 -1.0
references: [0] | citations: [1, 2, 3, 4, 5, 6, 7, 8, 9]
{1, 2, 3, 4, 5, 6, 7, 8, 9}
defaultdict(<class 'float'>, {2001: 2.0, 2002: 1.5, 2003: 1.3333333333333333, 2004: 1.25, 2005: 1.2, 2006: 1.1666666666666667, 2007: 1.1428571428571428, 2008: 1.125, 2009: 1.1111111111111112})

1 2001 4 8 8 -0.8888888888888888
references: [0, 1] | citations: [2, 3, 4, 5, 6, 7, 8, 9]
{0, 2, 3, 4, 5, 6, 7, 8, 9}
defaultdict(<class 'float'>, {2002: 1.5, 2003: 1.3333333333333333, 2004: 1.25, 2005: 1.2, 2006: 1.1666666666666667, 2007: 1.1428571428571428, 2008: 1.125, 2009: 1.1111111111111112})

2 2002 4 7 7 -0.875
references: [0, 1, 2] | citations: [3, 4, 5, 6, 7, 8, 9]
{0, 1, 3, 4, 5, 6, 7, 8, 9}
defaultdict(<class 'float'>, {2003: 1.3333333333333333, 2004: 1.25, 2005: 1.2, 2006: 1.1666666666666667, 2007: 1.1428571428571428, 2008: 1.125, 2009: 1.1111111111111112})

3 2003 4 6 6 -0.8571428571428571
references: [0, 1, 2, 3] | citations: [4, 5, 6, 7, 8, 9]
{0, 1, 2, 4, 5, 6, 7, 8, 9}
default

In [8]:
for aid in range(Nauthors):
    author = database.get_author(aid)
    print(author.start_year, author.end_year)
    print(author.career_length, author.total_productivity, author.annual_productivity)
    print("Total citations: {0}, H-index: {1}".format(author.total_citations, author.hindex))
    print()
print(author.cummulative_yearly_citations)
print(author.author_timeline)

2000 2008
9 7 0.7777777777777778
Total citations: 33, H-index: 4

2002 2009
8 6 0.75
Total citations: 18, H-index: 3

2000 2009
10 7 0.7
Total citations: 39, H-index: 5

{2000: 0.0, 2001: 1.0, 2002: 3.0, 2003: 6.0, 2004: 10.0, 2005: 15.0, 2006: 21.0, 2007: 27.0, 2008: 33.0, 2009: 39.0}
[[2000, 9], [2001, 8], [2002, 7], [2003, 6], [2004, 5], [2005, 4], [2009, 0]]


In [2]:
MAG = pyscisci.BibDatabase(name = 'mag2020')

pyscisci.dataset_interface.mag_interface.load_mag(MAG, path2files = '')

FileNotFoundError: [Errno 2] No such file or directory: 'mag/Affiliations.txt'